## < Main description of notebook >
#### Load special csv

In [17]:
import csv
import numpy as np
import gc
gc.collect()
reader = csv.reader(open("./meetup_sane.csv"), delimiter=",")
meetup = list(reader) 

#### Creating PANDAS dataframe

In [13]:
import pandas as pd 
mp = meetup[1:]
meetup_df = pd.DataFrame(mp, columns=meetup[0])

In [19]:
meetup_df


,member_id,bio,group_id,group_name,group_description,category_name,join_mode,members_count,group_rating,visibility,...,event_description,duration,fee_accepts,fee_amount,fee.required,yes_rsvp_count,venue_id,venue_name,venue_rating,venue_rating_count
0,81896,Why not?,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,Hello all - The October book will be: Between ...,10800,cash,2,0,20,11898,Heartland Cafe,3.79,275
1,81896,Why not?,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,"Hello all - As areminder, the November and Dec...",10800,cash,2,0,17,11898,Heartland Cafe,3.79,275
2,81896,Why not?,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,Hello all - The January book will be: Lincoln ...,10800,cash,2,0,17,11898,Heartland Cafe,3.79,275
3,81896,Why not?,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,Hello all - The February book will be: The Min...,10800,cash,2,0,15,11898,Heartland Cafe,3.79,275
4,81896,Why not?,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,Hello all - The March book will be: Hag-Seed M...,10800,cash,2,0,14,11898,Heartland Cafe,3.79,275
5,250609,Recent master's graduate looking to keep the c...,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,Hello all - The October book will be: Between ...,10800,cash,2,0,20,11898,Heartland Cafe,3.79,275
6,250609,Recent master's graduate looking to keep the c...,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,"Hello all - As areminder, the November and Dec...",10800,cash,2,0,17,11898,Heartland Cafe,3.79,275
7,250609,Recent master's graduate looking to keep the c...,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,Hello all - The January book will be: Lincoln ...,10800,cash,2,0,17,11898,Heartland Cafe,3.79,275
8,250609,Recent master's graduate looking to keep the c...,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,Hello all - The February book will be: The Min...,10800,cash,2,0,15,11898,Heartland Cafe,3.79,275
9,250609,Recent master's graduate looking to keep the c...,1687797,The Far Northside Co-Ed Book Club,There are several book clubs out there but the...,book clubs,open,350,4.66,public,...,Hello all - The March book will be: Hag-Seed M...,10800,cash,2,0,14,11898,Heartland Cafe,3.79,275


#### Implementing a RNN in tensorflow

In [23]:
gc.collect()


27

##### A LSTM based deep Siamese network for text similarity: 
###### Uses an character embedding layer, 
######  followed by a biLSTM and Energy Loss layer.

##### Usage : 
###### from siamese_network import SiameseLSTM


In [22]:
import tensorflow as tf
import numpy as np

class SiameseLSTM(object):
    
    def BiRNN(self, x, dropout, scope, embedding_size, sequence_length, hidden_units):
        n_hidden=hidden_units
        n_layers=3
        # Prepare data shape to match `static_rnn` function requirements
        x = tf.unstack(tf.transpose(x, perm=[1, 0, 2]))
        print(x)
        # Define lstm cells with tensorflow
        # Forward direction cell
        with tf.name_scope("fw"+scope),tf.variable_scope("fw"+scope):
            stacked_rnn_fw = []
            for _ in range(n_layers):
                fw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(fw_cell,output_keep_prob=dropout)
                stacked_rnn_fw.append(lstm_fw_cell)
            lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)

        with tf.name_scope("bw"+scope),tf.variable_scope("bw"+scope):
            stacked_rnn_bw = []
            for _ in range(n_layers):
                bw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(bw_cell,output_keep_prob=dropout)
                stacked_rnn_bw.append(lstm_bw_cell)
            lstm_bw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_bw, state_is_tuple=True)
        # Get lstm cell output

        with tf.name_scope("bw"+scope),tf.variable_scope("bw"+scope):
            outputs, _, _ = tf.nn.static_bidirectional_rnn(lstm_fw_cell_m, lstm_bw_cell_m, x, dtype=tf.float32)
        return outputs[-1]
    
    def contrastive_loss(self, y,d,batch_size):
        tmp= y *tf.square(d)
        #tmp= tf.mul(y,tf.square(d))
        tmp2 = (1-y) *tf.square(tf.maximum((1 - d),0))
        return tf.reduce_sum(tmp +tmp2)/batch_size/2
    
    def __init__(
        self, sequence_length, vocab_size, embedding_size, hidden_units, l2_reg_lambda, batch_size):

        # Placeholders for input, output and dropout
        self.input_x1 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x1")
        self.input_x2 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x2")
        self.input_y = tf.placeholder(tf.float32, [None], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0, name="l2_loss")
          
        # Embedding layer
        with tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                trainable=True,name="W")
            self.embedded_chars1 = tf.nn.embedding_lookup(self.W, self.input_x1)
            #self.embedded_chars_expanded1 = tf.expand_dims(self.embedded_chars1, -1)
            self.embedded_chars2 = tf.nn.embedding_lookup(self.W, self.input_x2)
            #self.embedded_chars_expanded2 = tf.expand_dims(self.embedded_chars2, -1)

        # Create a convolution + maxpool layer for each filter size
        with tf.name_scope("output"):
            self.out1=self.BiRNN(self.embedded_chars1, self.dropout_keep_prob, "side1", embedding_size, sequence_length, hidden_units)
            self.out2=self.BiRNN(self.embedded_chars2, self.dropout_keep_prob, "side2", embedding_size, sequence_length, hidden_units)
            self.distance = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(self.out1,self.out2)),1,keep_dims=True))
            self.distance = tf.div(self.distance, tf.add(tf.sqrt(tf.reduce_sum(tf.square(self.out1),1,keep_dims=True)),tf.sqrt(tf.reduce_sum(tf.square(self.out2),1,keep_dims=True))))
            self.distance = tf.reshape(self.distance, [-1], name="distance")
        with tf.name_scope("loss"):
            self.loss = self.contrastive_loss(self.input_y,self.distance, batch_size)
        #### Accuracy computation is outside of this class.
        with tf.name_scope("accuracy"):
            self.temp_sim = tf.subtract(tf.ones_like(self.distance),tf.rint(self.distance), name="temp_sim") #auto threshold 0.5
            correct_predictions = tf.equal(self.temp_sim, self.input_y)
            self.accuracy=tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [ ]:
import tensorflow as tf
import numpy as np

class SiameseLSTMw2v(object):
    """
    A LSTM based deep Siamese network for text similarity.
    Uses an word embedding layer (looks up in pre-trained w2v), followed by a biLSTM and Energy Loss layer.
    """
    
    def stackedRNN(self, x, dropout, scope, embedding_size, sequence_length, hidden_units):
        n_hidden=hidden_units
        n_layers=3
        # Prepare data shape to match `static_rnn` function requirements
        x = tf.unstack(tf.transpose(x, perm=[1, 0, 2]))
        # print(x)
        # Define lstm cells with tensorflow
        # Forward direction cell

        with tf.name_scope("fw"+scope),tf.variable_scope("fw"+scope):
            stacked_rnn_fw = []
            for _ in range(n_layers):
                fw_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0, state_is_tuple=True)
                lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(fw_cell,output_keep_prob=dropout)
                stacked_rnn_fw.append(lstm_fw_cell)
            lstm_fw_cell_m = tf.nn.rnn_cell.MultiRNNCell(cells=stacked_rnn_fw, state_is_tuple=True)

            outputs, _ = tf.nn.static_rnn(lstm_fw_cell_m, x, dtype=tf.float32)
        return outputs[-1]

    def contrastive_loss(self, y,d,batch_size):
        tmp= y *tf.square(d)
        #tmp= tf.mul(y,tf.square(d))
        tmp2 = (1-y) *tf.square(tf.maximum((1 - d),0))
        return tf.reduce_sum(tmp +tmp2)/batch_size/2
    
    def __init__(
        self, sequence_length, vocab_size, embedding_size, hidden_units, l2_reg_lambda, batch_size, trainableEmbeddings):

        # Placeholders for input, output and dropout
        self.input_x1 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x1")
        self.input_x2 = tf.placeholder(tf.int32, [None, sequence_length], name="input_x2")
        self.input_y = tf.placeholder(tf.float32, [None], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0, name="l2_loss")
          
        # Embedding layer
        with tf.name_scope("embedding"):
            self.W = tf.Variable(
                tf.constant(0.0, shape=[vocab_size, embedding_size]),
                trainable=trainableEmbeddings,name="W")
            self.embedded_words1 = tf.nn.embedding_lookup(self.W, self.input_x1)
            self.embedded_words2 = tf.nn.embedding_lookup(self.W, self.input_x2)
        print self.embedded_words1
        # Create a convolution + maxpool layer for each filter size
        with tf.name_scope("output"):
            self.out1=self.stackedRNN(self.embedded_words1, self.dropout_keep_prob, "side1", embedding_size, sequence_length, hidden_units)
            self.out2=self.stackedRNN(self.embedded_words2, self.dropout_keep_prob, "side2", embedding_size, sequence_length, hidden_units)
            self.distance = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(self.out1,self.out2)),1,keep_dims=True))
            self.distance = tf.div(self.distance, tf.add(tf.sqrt(tf.reduce_sum(tf.square(self.out1),1,keep_dims=True)),tf.sqrt(tf.reduce_sum(tf.square(self.out2),1,keep_dims=True))))
            self.distance = tf.reshape(self.distance, [-1], name="distance")
        with tf.name_scope("loss"):
            self.loss = self.contrastive_loss(self.input_y,self.distance, batch_size)
        #### Accuracy computation is outside of this class.
        with tf.name_scope("accuracy"):
            self.temp_sim = tf.subtract(tf.ones_like(self.distance),tf.rint(self.distance), name="temp_sim") #auto threshold 0.5
            correct_predictions = tf.equal(self.temp_sim, self.input_y)
            self.accuracy=tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [33]:
import re
s = re.sub('[^0-9a-zA-Z]+', '', "123 asf jogrs")

In [34]:
s

'123asfjogrs'

In [35]:
{1:"a"}

{1: 'a'}